In [ ]:
import pyvista as pv
import numpy as np 
import os
import pandas as pd 
import pickle
import seaborn as sns
import scipy.stats as stats
import scipy.io as sio
import random as rd 
import pyvista as pv 
import matplotlib.pyplot as plt 
from scipy.spatial import distance
from nilearn.datasets import fetch_surf_fsaverage
from nilearn import surface

# Utils import
from utils.mathutils import compute_vertex_normals, build_template_adjacency_two_hemis, compute_mean_curvature, compute_curvature_differences, compute_hausdorff_metrics, compute_point_distances, compute_normal_differences
from utils.file_manip.Matlab_to_array import load_faces, load_vertices
from utils.cortical import surface_preprocess as sp
from utils.cortical import spherical_harmonics as SH
from src.cortical_transformation.reconstruction import reconstruct_brain
from utils.mathutils import cart_to_sph
from utils.file_manip.vtk_processing import convert_triangles_to_pyvista

In [ ]:
data_path=r"C:\Users\wbou2\Desktop\meg_to_surface_ml\src\cortical_transformation\data"
main_folder = r"C:\Users\wbou2\Desktop\meg_to_surface_ml\data\Anatomy_data_CAM_CAN"
template_projection_lh = np.load(os.path.join(data_path, "lh_sphere_projection.npz"))
template_projection_rh = np.load(os.path.join(data_path, "rh_sphere_projection.npz"))


sub_name="sub-CC110033"
sub_file=os.path.join(main_folder, sub_name)

In [ ]:
lmax = 80
sigma=1e-7
lambda_reg=1e-7
n_subjects=len(os.listdir(main_folder))

In [ ]:
# Load hemisphere-specific harmonics and slice according to lmax (limited to lmax<=80)
Y_lh_full = np.load(os.path.join(data_path, "Y_lh.npz"))['Y']
Y_rh_full = np.load(os.path.join(data_path, "Y_rh.npz"))['Y']

# Left hemisphere uses first part, right hemisphere uses second part
Y_lh = Y_lh_full[:, :(lmax+1)**2]
Y_rh = Y_rh_full[:, :(lmax+1)**2:]

In [ ]:
# Load fsaverage
fsaverage7 = fetch_surf_fsaverage(mesh='fsaverage7')
surf_lh = surface.load_surf_mesh(fsaverage7['pial_left'])
surf_rh = surface.load_surf_mesh(fsaverage7['pial_right'])

# Output file 
folder_output = r"C:\Users\wbou2\Desktop\meg_to_surface_ml\data\fsaverage"
os.makedirs(folder_output, exist_ok=True)

# Left Hemi
surface_mesh_lh = (surf_lh[0], surf_lh[1])
coords_lh, tris_lh = sp.get_resampled_inner_surface(surface_mesh_lh, 'lh')
center_lh = np.mean(coords_lh, axis=0)
coords_lh = coords_lh - center_lh
output_file_lh = os.path.join(folder_output, "lh_resampled.npz")
np.savez(output_file_lh, coords=coords_lh, tris=tris_lh, center=center_lh)

# Right hemi
surface_mesh_rh = (surf_rh[0], surf_rh[1])
coords_rh, tris_rh = sp.get_resampled_inner_surface(surface_mesh_rh, 'rh')
center_rh = np.mean(coords_rh, axis=0)
coords_rh = coords_rh - center_rh
output_file_rh = os.path.join(folder_output, "rh_resampled.npz")
np.savez(output_file_rh, coords=coords_rh, tris=tris_rh, center=center_rh)

In [ ]:
# Save resampled cortical surfaces
for folder in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, folder)
    if os.path.isdir(folder_path):
        print(f"Processing subject: {folder}")
        try:
            left_vertices_file = os.path.join(folder_path, "lh_vertices.mat")
            left_faces_file = os.path.join(folder_path, "lh_faces.mat")
            output_file = os.path.join(folder_path, "lh_resampled.npz")
            
            left_faces = load_faces(left_faces_file)
            left_vertices = load_vertices(left_vertices_file)
            coords, tris = sp.get_resampled_inner_surface((left_vertices, left_faces), 'lh')
            center = np.mean(coords, axis=0)
            coords = coords - center
            np.savez(output_file, coords=coords, tris=tris, center=center)
            print(f"  Left hemisphere processed successfully")

        except Exception as e:
            print(f"  Error processing left hemisphere: {str(e)}")
        
        try:
            right_vertices_file = os.path.join(folder_path, "rh_vertices.mat")
            right_faces_file = os.path.join(folder_path, "rh_faces.mat")
            output_file = os.path.join(folder_path, "rh_resampled.npz")
            
            right_faces = load_faces(right_faces_file)
            right_vertices = load_vertices(right_vertices_file)
            coords, tris = sp.get_resampled_inner_surface((right_vertices, right_faces), 'rh')
            center = np.mean(coords, axis=0)
            coords = coords - center
            np.savez(output_file, coords=coords, tris=tris, center=center)
            print(f"  Right hemisphere processed successfully")
        except Exception as e:
            print(f"  Error processing right hemisphere: {str(e)}")

In [ ]:
#Save coefficients for our template surface 

fsaverage_path=r"C:\Users\wbou2\Desktop\meg_to_surface_ml\data\fsaverage"
# Load resampled data
fsav_lh = np.load(os.path.join(fsaverage_path, "lh_resampled.npz"))
fsav_rh = np.load(os.path.join(fsaverage_path, "rh_resampled.npz"))

# Prepare resampled surfaces for coefficient computation
resampled_lh = (fsav_lh['coords'], fsav_lh['tris'])
resampled_rh = (fsav_rh['coords'], fsav_rh['tris'])

# Compute coefficients for both hemispheres
coeffs_fsav_lh = SH.compute_coefficients_SVD(Y_lh, resampled_lh, lmax, lambda_reg=lambda_reg)
coeffs_fsav_rh = SH.compute_coefficients_SVD(Y_rh, resampled_rh, lmax, lambda_reg=lambda_reg)

# Save coefficients
with open(os.path.join(fsaverage_path, "coeffs_lh.pkl"), 'wb') as f:
    pickle.dump(coeffs_fsav_lh, f)
with open(os.path.join(fsaverage_path, "coeffs_rh.pkl"), 'wb') as f:
    pickle.dump(coeffs_fsav_rh, f)

In [ ]:
# Save coefficients for each hemisphere of each subject
for folder in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, folder)
    if os.path.isdir(folder_path):
        coeffs_lh_path = os.path.join(folder_path, "coeffs_lh.pkl")
        coeffs_rh_path = os.path.join(folder_path, "coeffs_rh.pkl")
        
        # Load hemisphere-specific harmonics and slice according to lmax
        Y_lh_full = np.load(os.path.join(data_path, "Y_lh.npz"))['Y']
        Y_rh_full = np.load(os.path.join(data_path, "Y_rh.npz"))['Y']
        
        # Left hemisphere uses first part, right hemisphere uses second part
        Y_lh = Y_lh_full[:, :(lmax+1)**2]
        Y_rh = Y_rh_full[:, :(lmax+1)**2:]
        
        print(f"Processing left hemi of {folder}")
        left_resampled_data = np.load(os.path.join(folder_path, "lh_resampled.npz"))
        
        # Smooth the left hemisphere surface
        left_smoothed_coords = sp.smooth_surface(left_resampled_data['coords'], 
                                           left_resampled_data['tris'],
                                           n_iterations=5, 
                                           relaxation_factor=0.5)
        
        coeffs_lh = SH.compute_coefficients_SVD(Y_lh, 
                                              (left_smoothed_coords, left_resampled_data['tris']), 
                                              lmax, 
                                              lambda_reg)
        with open(coeffs_lh_path, 'wb') as f:
            pickle.dump(coeffs_lh, f)
        
        print(f"Processing right hemi of {folder}")
        right_resampled_data = np.load(os.path.join(folder_path, "rh_resampled.npz"))
        
        # Smooth the right hemisphere surface
        right_smoothed_coords = sp.smooth_surface(right_resampled_data['coords'], 
                                            right_resampled_data['tris'],
                                            n_iterations=5, 
                                            relaxation_factor=0.5)
        
        coeffs_rh = SH.compute_coefficients_SVD(Y_rh,
                                              (right_smoothed_coords, right_resampled_data['tris']),
                                              lmax, 
                                              lambda_reg)
        with open(coeffs_rh_path, 'wb') as f:
            pickle.dump(coeffs_rh, f)

In [ ]:
for folder in os.listdir(main_folder):
   folder_path = os.path.join(main_folder, folder)
   if os.path.isdir(folder_path):
       # Load centers from resampled files
       lh_data = np.load(os.path.join(folder_path, "lh_resampled.npz"))
       rh_data = np.load(os.path.join(folder_path, "rh_resampled.npz"))
       
       # Get centers
       lh_center = lh_data['center']
       rh_center = rh_data['center']
       
       # Save centers
       np.savez(os.path.join(folder_path, "lh_center.npz"), center=lh_center)
       np.savez(os.path.join(folder_path, "rh_center.npz"), center=rh_center)